# 1. General Settings

### 1.1 Import libraries

In [1]:
### bugs for transformers(4.28.1), update to newest version 4.31.0
from dataclasses import dataclass, field
from typing import Optional
import torch
import re
import numpy as np
import random
from accelerate import Accelerator
from datasets import load_dataset
import pandas as pd
import json
import os
import sys
from typing import Dict, List
import time

from tqdm import tqdm
from transformers import Adafactor, AutoTokenizer, HfArgumentParser, pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration

from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel,TaskType

from transformers import T5Tokenizer, T5ForConditionalGeneration



[2023-09-30 23:58:28,989] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


### 1.2 Random Seed

In [2]:
def set_seed(seed: int):
    """
   A function to fix random seed in `numpy`,`random`,  and `torch`.

    Args:
        seed (`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
set_seed(1000)

### 1.3 Metrics

In [3]:
# Reference: https://www.nltk.org/

import operator
from functools import reduce
from math import fabs
from random import shuffle

try:
    from scipy.stats.stats import betai
except ImportError:
    betai = None

from nltk.util import LazyConcatenation, LazyMap



def accuracy(reference, test):
    """
    Given a list of reference values and a corresponding list of test
    values, return the fraction of corresponding values that are
    equal.  In particular, return the fraction of indices
    ``0<i<=len(test)`` such that ``test[i] == reference[i]``.

    :type reference: list
    :param reference: An ordered list of reference values.
    :type test: list
    :param test: A list of values to compare against the corresponding
        reference values.
    :raise ValueError: If ``reference`` and ``length`` do not have the
        same length.
    """
    if len(reference) != len(test):
        raise ValueError("Lists must have the same length.")
    return sum(float(x) == float(y) for x, y in zip(reference, test) if y != '[invalid]' ) / len(test)


def precision(reference, test):
    """
    Given a set of reference values and a set of test values, return
    the fraction of test values that appear in the reference set.
    In particular, return card(``reference`` intersection ``test``)/card(``test``).
    If ``test`` is empty, then return None.

    :type reference: set
    :param reference: A set of reference values.
    :type test: set
    :param test: A set of values to compare against the reference set.
    :rtype: float or None
    """
    if not hasattr(reference, "intersection") or not hasattr(test, "intersection"):
        raise TypeError("reference and test should be sets")

    if len(test) == 0:
        return None
    else:
        return len(reference.intersection(test)) / len(test)



def recall(reference, test):
    """
    Given a set of reference values and a set of test values, return
    the fraction of reference values that appear in the test set.
    In particular, return card(``reference`` intersection ``test``)/card(``reference``).
    If ``reference`` is empty, then return None.

    :type reference: set
    :param reference: A set of reference values.
    :type test: set
    :param test: A set of values to compare against the reference set.
    :rtype: float or None
    """
    if not hasattr(reference, "intersection") or not hasattr(test, "intersection"):
        raise TypeError("reference and test should be sets")

    if len(reference) == 0:
        return None
    else:
        return len(reference.intersection(test)) / len(reference)



def f_measure(reference, test, alpha=0.5):
    """
    Given a set of reference values and a set of test values, return
    the f-measure of the test values, when compared against the
    reference values.  The f-measure is the harmonic mean of the
    ``precision`` and ``recall``, weighted by ``alpha``.  In particular,
    given the precision *p* and recall *r* defined by:

    - *p* = card(``reference`` intersection ``test``)/card(``test``)
    - *r* = card(``reference`` intersection ``test``)/card(``reference``)

    The f-measure is:

    - *1/(alpha/p + (1-alpha)/r)*

    If either ``reference`` or ``test`` is empty, then ``f_measure``
    returns None.

    :type reference: set
    :param reference: A set of reference values.
    :type test: set
    :param test: A set of values to compare against the reference set.
    :rtype: float or None
    """
    p = precision(reference, test)
    r = recall(reference, test)
    if p is None or r is None:
        return None
    if p == 0 or r == 0:
        return 0
    return 1.0 / (alpha / p + (1 - alpha) / r)


### 1.4 Prompt format/ examples

In [4]:
svamp_format = '{} Let’ s think step by step.'

### 1.5 Dataset

In [24]:
def build_test_dataset(
    dataset_name="/root/autodl-tmp/SVAMP",prompt_examples=svamp_format,prompt_format='default'
):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.
        prompt_examples ('str'):
            The few shot examples for testing
        prompt_format ('str'):
            The format of the prompting for testing

    Returns:
        dataset for test of checkpoints of llms
    """


    ds_test = load_dataset(dataset_name, split="test")
    original_columns = ds_test.column_names


    def preprocess_function(examples):

        body=examples["Body"]
        question=examples["Question"]
        if prompt_format=='default':
            full_question=body +' '+ question
            query = '{} Let’ s think step by step, then give the answer after "the answer is".'.format(full_question.replace('\n', ' '))

        return {"query": query,"full_question":full_question }
    

    ds_test = ds_test.map(
        preprocess_function,
        batched=False,
        #remove_columns=original_columns,
    )


    print(ds_test)
    return ds_test

In [25]:
# We retrieve the dataloader by calling the `build_dataset` function.
dataset = build_test_dataset(dataset_name="/root/autodl-tmp/msc_ml/llm_thought/Dataset/SVAMP")
prompt_test=dataset["query"]
questions_test=dataset["full_question"]
answer_test=dataset["Answer"]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Dataset({
    features: ['Body', 'Question', 'ID', 'Type', 'Equation', 'Answer', 'query', 'full_question'],
    num_rows: 300
})


In [26]:
prompt_test[1]

'Paul got a box of some crayons for his birthday. During the school year he gave 52 crayons to his friends while he lost 535 crayons. If he only had 492 crayons left How many crayons had been lost or given away? Let’ s think step by step, then give the answer after "the answer is".'

### 1.6 Function to Process Responses

In [43]:
ANS_RE = re.compile(r"Answer: .*?(\$?)(\-?[0-9\.\,]+)")
ANS_RE = re.compile(r"answer:\s*?([\-]?[0-9]+(?:\.[0-9]+)?)(?![0-9])", re.IGNORECASE)
ANS_RE = re.compile(r"answer:\s*?([\-]?[0-9]+(?:\.[0-9]+)?)(?![0-9])", re.IGNORECASE)




INVALID_ANS = "[invalid]"
# extract the numerical answer for arithmetic dataset of the default prompt format
def _extract_answer(completion: str):
    match = ANS_RE.search(completion)
    if match:
        match_str = match.group(1).strip()
        match_str = match_str.replace(",", "")
        return match_str
    else:
        return INVALID_ANS


### 1.7 Tokenizer

In [28]:
# tokenizer for llama
model_name='/root/autodl-tmp/flan-t5-large'
tokenizer = AutoTokenizer.from_pretrained("/root/autodl-tmp/flan-t5-large")


### 1.8 Generate responses to questions

In [29]:
"""
def generate_responses(model,tokenizer,prompt_test=prompt_test,answer_test=answer_test,test_epochs=1,batch_size=2):


    # because of sampling of tokens, we can increase epochs of test for more precise evaluation
    test_epochs=test_epochs

    outputs_text=[]
    # the prompt and answer correspondong to output_text
    prompts_for_outputs=[]
    answers_for_outputs=[]

    batch_size = batch_size  

    for epoch in tqdm(range(test_epochs)):
        for i in tqdm(range(0, len(prompt_test), batch_size)):  # Increment by batch_size

            # Slice the data to create a batch
            batch_questions = prompt_test[i:i+batch_size]
            batch_answers = answer_test[i:i+batch_size]

            batch_inputs = tokenizer(batch_questions, padding=True, return_tensors="pt", truncation=True, max_length=2048).to('cuda')

            batch_outputs = model.generate(**batch_inputs,    
                                     max_new_tokens=256,
                                     top_k=50,
                                     temperature=1.0,
                                     top_p=0.95,
                                     do_sample=True,
                                     repetition_penalty=1.1)
            batch_input_length = batch_inputs.input_ids.shape[1]
            batch_generated_tokens = batch_outputs[:, batch_input_length:]
            batch_output_text = tokenizer.batch_decode(batch_generated_tokens)
            print(batch_output_text)
        
        
            outputs_text.extend(batch_output_text)
            prompts_for_outputs.extend(batch_questions)
            answers_for_outputs.extend(batch_answers)
    return outputs


outputs=generate_responses(model=base_model,tokenizer=tokenizer)
"""

'\ndef generate_responses(model,tokenizer,prompt_test=prompt_test,answer_test=answer_test,test_epochs=1,batch_size=2):\n\n\n    # because of sampling of tokens, we can increase epochs of test for more precise evaluation\n    test_epochs=test_epochs\n\n    outputs_text=[]\n    # the prompt and answer correspondong to output_text\n    prompts_for_outputs=[]\n    answers_for_outputs=[]\n\n    batch_size = batch_size  \n\n    for epoch in tqdm(range(test_epochs)):\n        for i in tqdm(range(0, len(prompt_test), batch_size)):  # Increment by batch_size\n\n            # Slice the data to create a batch\n            batch_questions = prompt_test[i:i+batch_size]\n            batch_answers = answer_test[i:i+batch_size]\n\n            batch_inputs = tokenizer(batch_questions, padding=True, return_tensors="pt", truncation=True, max_length=2048).to(\'cuda\')\n\n            batch_outputs = model.generate(**batch_inputs,    \n                                     max_new_tokens=256,\n              

In [30]:
def generate_responses(generator,prompt_test=prompt_test,answer_test=answer_test,questions_test=prompt_test,test_epochs=1,batch_size=10):
    generator=generator

    # if using sampling of tokens, we can increase epochs of test for more precise evaluation(law of large numbers)
    test_epochs=test_epochs

    outputs=[]
    prompts_for_outputs=[]
    questions_for_outputs=[]
    answers_for_outputs=[]
    

    batch_size = batch_size  

    for epoch in tqdm(range(test_epochs)):
        for i in tqdm(range(0, len(prompt_test), batch_size)):  # Increment by batch_size

            # Slice the data to create a batch
            batch_prompts = prompt_test[i:i+batch_size]
            batch_questions=questions_test[i:i+batch_size]
            batch_answers = answer_test[i:i+batch_size]

            batch_generations = generator(batch_prompts,batch_size=batch_size,return_tensors=False)
            print(batch_generations)
            outputs.extend([gen['generated_text'] for gen in batch_generations])
            prompts_for_outputs.extend(batch_prompts)
            questions_for_outputs.extend(batch_questions)
            answers_for_outputs.extend(batch_answers)
    return outputs,questions_for_outputs,answers_for_outputs,prompts_for_outputs

# 2.base model

### 2.1 Load base model and builld pipeline

In [34]:
# load base model
model_name="/root/autodl-tmp/flan-t5-large"
base_model = T5ForConditionalGeneration.from_pretrained(model_name,torch_dtype=torch.float16,
                                                  device_map='auto')

In [36]:
gen_kwargs=dict(
    max_new_tokens=521,
    top_k=50,
    temperature=1.0,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.1
)


# Create the pipeline for base model. To make results comparable, we use greedy search to eliminate the influence of sampling
base_generator= pipeline("text2text-generation", model=base_model, tokenizer=tokenizer,
                         max_new_tokens=512,
                         top_k=50,
                         do_sample=False,
                         temperature=1.0,
                         top_p=0.95,
                         eos_token_id= tokenizer.eos_token_id,
                         device_map='auto'
                        )

### 2.1 Load base model and builld pipeline### 2.2 Generate reponses by base model

In [37]:
base_responses,base_response_questions,base_response_answers,base_prompts=generate_responses(base_generator,test_epochs=4,batch_size=10)

  0%|          | 0/30 [00:00<?, ?it/s]Both `max_new_tokens` (=512) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)

  3%|▎         | 1/30 [01:12<35:02, 72.50s/it]Both `max_new_tokens` (=512) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'Mary needs to add 6 - 5 = 2 cups of sugar. She needs to add 7 - 7 = 2 cups of salt. Mary needs to add 2 cups of sugar - 2 cups of salt = 2 cups of flour. The answer: 2.'}, {'generated_text': 'He had 492 - 52 = 312 crayons left. He lost 535 - 52 = 312 crayons. So, 312 - 312 = 126 crayons were lost or given away. The answer: 126.'}, {'generated_text': "Robin's hair was 16 - 11 = 10 inches long. He cut off 11 - 2 = 10 inches. So his hair is 10 + 12 = 18 inches long. The answer is 18."}, {'generated_text': 'Josh lost 9 - 23 = 2 marbles. So he lost 2 - 9 = 1 more marble than those he found. The answer: 1.'}, {'generated_text': 'Ed had 29 + 17 = 41 marbles. Doug had 41 - 17 = 43 marbles. Ed had 43 - 43 = 21 more marbles than Doug. The answer is 21.'}, {'generated_text': 'Paco had 25 - 5 = 20 cookies left. He bought 3 + 3 = 6 cookies. Paco ate 20 - 6 = 10 more cookies than those he bought. The answer is 10.'}, {'generated_text': 'Paul sold 71 books so he bought 71 - 38 = 

  0%|          | 0/4 [01:35<?, ?it/s]


KeyboardInterrupt: 

### 2.3 Process Responses for base_model

In [42]:
base_extracted_responses = [_extract_answer(response) for response in base_responses]

print('invalid rate:',sum(np.array(base_extracted_responses)=='[invalid]')/len(base_extracted_responses))
np.array(base_responses)[np.array(base_extracted_responses)=='[invalid]']
#extracted_responses
#_extract_answer("Answer: $-12.1333,1 and 2,2")

invalid rate: 0.49


array(['Paul had 492 - 52 = 312 crayons left. Paul lost 535 - 52 = 426 crayons. So, 312 - 426 = 126 crayons were lost or given away. Thus, the answer is 126.',
       'Josh lost 9 - 23 = 2 marbles. So he lost 2 - 9 = 1 more marble than those he found. So the answer is 1.',
       'Paco ate 5 cookies so he had 25 - 5 = 20 cookies left. He bought 3 cookies so he had 20 + 3 = 23 cookies. Paco ate 20 cookies so he had 23 - 20 = 9 cookies left. He ate 9 cookies so he had 9 - 3 = 6 more cookies. So, the answer is 6.',
       'The grocery store had 72 - 32 = 28 bottles of regular soda. The grocery store had 78 - 78 = 8 bottles of diet soda. The grocery store had 8 - 32 = 4 bottles of regular soda. The grocery store had 8 - 4 = 4 bottles of diet soda. The grocery store had 8 - 4 = 4 bottles of regular soda. The grocery store had 4 bottles of regular soda and 8 bottles of diet soda. The grocery store had 4 bottles of regular soda and 8 bottles of diet soda. The grocery store had 4 bottles of re

### 2.4 Record the results for Base Model

In [17]:
import pandas as pd

# Create a DataFrame with named columns
df_base_svamp = pd.DataFrame({'Prompt':base_prompts,'Question': base_response_questions, 'Response': base_responses,'extracted_response':base_extracted_responses, 'Answer': base_response_answers})

# Write to CSV
df_base_svamp.to_csv('test_zeroshot_base_svamp.csv', index=True)

### 2.5 calculate metrics for base model

In [18]:
svamp_base_accuracy=accuracy(reference=base_response_answers, test=base_extracted_responses)

In [19]:
svamp_base_accuracy

0.4533333333333333

# 3.fine-tuned model

### 3.1 Load fine-tuned model and builld pipeline

In [13]:
# load fine-tuned model
torch.cuda.empty_cache()
new_model_name="/root/autodl-tmp/msc_ml/llama_2/ckpts_svamp/llama_2_svamp_8_20"
new_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                 torch_dtype=torch.float16,
                                                 device_map=0,
                                                )

new_model = PeftModel.from_pretrained(new_model, new_model_name)
new_model = new_model.merge_and_unload()

#model.add_adapter(lora_config, adapter_name="adapter_1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [14]:
# Create the pipeline for new model
new_generator= pipeline("text-generation", model=new_model, tokenizer=tokenizer,
                         max_new_tokens=512,
                         #top_k=50,
                         temperature=1.0,
                         #top_p=0.95,
                         do_sample=False,
                         #repetition_penalty=1.1,
                         #device=0,
                       )

### 3.2 Generate reponses by fine-tuned model

In [15]:
new_responses,new_response_questions,new_response_answers,new_prompts=generate_responses(new_generator,batch_size=10)

  0%|          | 0/30 [00:00<?, ?it/s]/root/miniconda3/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(

  3%|▎         | 1/30 [00:11<05:40, 11.74s/it]

[[{'generated_text': "  Sure, I'd be happy to help! Here are my thoughts on the question:\nThoughts:\n1. Let's start by identifying the given information:\n* Mary has already added 5 cups of flour.\n* The recipe calls for 6 cups of flour.\n* The recipe also calls for 8 cups of sugar.\n* The recipe calls for 7 cups of salt.\n2. Now, we need to find out how many more cups of sugar Mary needs to add than cups of salt. Let's do this by subtracting the amount of flour she has already added from the total amount of flour needed:\n6 cups of flour - 5 cups of flour = 1 cup of flour left to add.\n3. Since the recipe calls for 8 cups of sugar, and Mary has already added 5 cups of flour, she needs to add 3 more cups of sugar (8 - 5 = 3).\n4. Now, let's find out how many more cups of salt Mary needs to add than cups of sugar she needs to add:\n7 cups of salt - 3 cups of sugar = 4 cups of salt left to add.\n5. Therefore, Mary needs to add 4 cups of salt.\nAnswer: 4 cups of salt."}], [{'generated_te


  7%|▋         | 2/30 [00:19<04:30,  9.65s/it]

[[{'generated_text': '\nThoughts:\n\n* Step 1: Identify the given information: Lewis earns $403 every week during 233 weeks of harvest, and he has to pay $49 rent every week.\n* Step 2: Calculate the total amount of money Lewis earns during harvest season: 233 weeks x $403 per week = $93,530.\n* Step 3: Subtract the amount of rent Lewis has to pay each week: $93,530 - $49 per week = $89,440.\n* Step 4: Calculate the total amount of money Lewis earns during harvest season after subtracting the rent: $89,440.\nAnswer: $89,440.'}], [{'generated_text': "\nThoughts:\nLet's start by identifying the information given in the question:\n\n* Jessie weighed 92 kilograms initially.\n* She lost 56 kilograms in the first week of jogging.\n* She lost 99 kilograms in the second week of jogging.\n\nNow, let's calculate Jessie's weight after the first week of jogging:\n\n* Initial weight = 92 kg\n* Loss in first week = 56 kg\n* New weight = Initial weight - Loss = 92 kg - 56 kg = 36 kg\n\nSo, Jessie wei


 10%|█         | 3/30 [00:28<04:12,  9.36s/it]

...

[[{'generated_text': "\nThoughts:\n\n1. Let's start by identifying the information given in the question:\n\t* Mom buys 28 packages.\n\t* Each package contains 56 white t-shirts in total.\n2. To find out how many white t-shirts each package has, we need to divide the total number of t-shirts in each package by the number of packages:\n\t* 56 white t-shirts / 28 packages =?\n3. Now, let's simplify the calculation:\n\t* 56 / 28 = 2\n4. So, each package has 2 white t-shirts.\nAnswer: 2"}], [{'generated_text': '\nThoughts:\n\n1. Dan spent $ 7 on the candy bar.\n2. Dan spent $ 6 on the chocolate.\n3. Total amount spent = $ 7 + $ 6 = $ 13.\n\nAnswer: $ 13'}], [{'generated_text': '\nThoughts:\n\n1. Danny had 25 bottle caps in his collection earlier.\n2. He found 32 bottle caps at the park.\n3. To find out how many more bottle caps he found, we need to subtract the number he already had from the number he found.\n4. 32 - 25 = 7\n\nAnswer: 7'}], [{'generated_text': "  Great! Let's work through 


 97%|█████████▋| 29/30 [04:42<00:08,  8.32s/it]

[[{'generated_text': '\nThoughts:\n\n1. Dan spent $2 to buy a candy bar.\n2. He spent $3 to buy a chocolate.\n3. To find out how much more he spent on the chocolate than the candy bar, we need to subtract the cost of the candy bar from the cost of the chocolate.\n4. So, the difference between the cost of the chocolate and the candy bar is $3 - $2 = $1.\nAnswer: $1'}], [{'generated_text': "\nThoughts:\n\n1. Let's start by identifying the information given in the question:\n\t* Tiffany had 7 bags of cans on Monday.\n\t* The next day, she found 12 more bags worth of cans.\n2. To find out how many more bags Tiffany found on the next day than she had on Monday, we need to subtract the number of bags she had on Monday from the number of bags she found on the next day:\n\t* 12 bags - 7 bags = 5 bags\nSo, Tiffany found 5 more bags of cans on the next day than she had on Monday.\nAnswer: 5"}], [{'generated_text': "\nThoughts:\nLet's start by identifying the information given in the question:\n*


100%|██████████| 1/1 [04:57<00:00, 297.13s/it]

[[{'generated_text': "\nThoughts:\n\n1. Let's start by counting the number of bottles of regular soda: 22.\n2. Now, let's count the number of bottles of diet soda: 61.\n3. To find out how many more bottles of diet soda there are than regular soda, we can subtract the number of regular soda bottles from the number of diet soda bottles: 61 - 22 = 39.\n4. So, the grocery store has 39 more bottles of diet soda than regular soda.\nAnswer: 39"}], [{'generated_text': "\nThoughts:\nLet's start by identifying the information given in the question:\n* Paul got a box of crayons for his birthday.\n* By the end of the school year, he had either lost or given away 551 of them.\n* He only had 177 left.\nNow, let's think about how we can use this information to find the number of crayons Paul got for his birthday.\n1. If Paul had 551 crayons at the end of the school year, and he gave away or lost 551, then he must have had more crayons at the beginning of the school year.\n2. If he had more crayons at

### 3.3 Process Responses for new_model

In [16]:
new_extracted_responses = [_extract_answer(response) for response in new_responses]

print('invalid rate:',sum(np.array(new_extracted_responses)=='[invalid]')/len(new_extracted_responses))
np.array(new_responses)[np.array(new_extracted_responses)=='[invalid]']
#extracted_responses
#_extract_answer("Answer: $-12.1333,1 and 2,2")

invalid rate: 0.006666666666666667


array(["\nThoughts:\nLet's start by using the information given in the question to set up an equation. We know that the grasshopper jumped 39 inches, and the frog jumped x inches. We also know that the mouse jumped y inches, where y is the distance the mouse jumped.\nWe can start by using the fact that the grasshopper jumped 19 inches farther than the frog to write:\n39 = x + 19\nNow, we can use the fact that the mouse jumped 12 inches less than the frog to write:\nx + 12 = y\nNow we have two equations and two unknowns. Can you use these equations to find the value of y, which is the distance the mouse jumped?",
       "\nThoughts:\nLet's start by dividing the total number of skittles and erasers by the number of groups.\nTotal number of skittles: 4502\nTotal number of erasers: 4276\nNumber of groups: 154\nNow, we can find the size of each group by dividing the total number of skittles and erasers by the number of groups.\nSize of each group:\nSkittles: 4502 / 154 = 29\nErasers: 4276 /

### 3.4 Record the results for Base Model

In [17]:
import pandas as pd

# Create a DataFrame with named columns
df_new_svamp = pd.DataFrame({'Prompt':new_prompts,'Question': new_response_questions, 'Response': new_responses,'extracted_response':new_extracted_responses, 'Answer': new_response_answers})

# Write to CSV
df_new_svamp.to_csv('test_zeroshot_new_svamp.csv', index=True)

### 3.5 calculate metrics for base model

In [20]:
svamp_new_accuracy=accuracy(reference=new_response_answers, test=new_extracted_responses)

In [21]:
svamp_new_accuracy

0.53

# Bug fix

In [13]:
import rich
from rich.table import Table
rich_table = Table(*["prompt"], title='1', show_lines=True)
for ix in range(2):
    rich_table.add_row("""[/INST][INST]""")
rich_table

In [44]:
ds_all = load_dataset("/root/autodl-tmp/SVAMP", split="all")

In [45]:
ds_train = load_dataset("/root/autodl-tmp/SVAMP", split="train")

In [48]:
sum(np.array(ds_train['Body'])==np.array(ds_all['Body'][:700]))

700

In [61]:
ds_train['Answer']

[145.0,
 19.0,
 3.0,
 198.0,
 63.0,
 322.0,
 30.0,
 192.0,
 26.0,
 17.0,
 1.0,
 20.0,
 4.0,
 2.0,
 7.0,
 70.0,
 345.0,
 266.0,
 54.0,
 807.0,
 3.0,
 12.0,
 23.0,
 31.0,
 1201565.0,
 89.0,
 574.0,
 8.0,
 146.0,
 1.0,
 14.0,
 2.0,
 7.0,
 12.0,
 21.0,
 14.0,
 2.0,
 51.0,
 2.0,
 15.0,
 20.0,
 3.0,
 7.0,
 42.0,
 756.0,
 11.0,
 5.0,
 12.0,
 65.0,
 1.0,
 111.0,
 1.0,
 27.0,
 4.0,
 17.0,
 13.0,
 10.0,
 77.0,
 1092.0,
 5.0,
 7.0,
 45.0,
 10.0,
 52.0,
 2.0,
 6.0,
 16.0,
 469.0,
 12518.0,
 1.0,
 2.0,
 131.0,
 92.0,
 640.0,
 42.0,
 7.0,
 25.0,
 1.0,
 2.0,
 11.0,
 15.0,
 20.0,
 8722.0,
 5.0,
 83.0,
 29.0,
 53.0,
 3.0,
 33.0,
 1.0,
 16.0,
 47.0,
 5590.0,
 1.0,
 1.0,
 49.0,
 9.0,
 78.0,
 34.0,
 7.0,
 24.0,
 34.0,
 10.0,
 14.0,
 10.0,
 15.0,
 621.0,
 1.0,
 2.0,
 14.0,
 8.0,
 2.0,
 14.0,
 89.0,
 20.0,
 7.0,
 1.0,
 16.0,
 21.0,
 3.0,
 5.0,
 9.0,
 76.0,
 28.0,
 30.0,
 19.0,
 2.0,
 22.0,
 7.0,
 42.0,
 14.0,
 24.0,
 8.0,
 27.0,
 31.0,
 2.0,
 21.0,
 4.0,
 80.0,
 3.0,
 2.0,
 7.0,
 1.0,
 4.0,
 4.0,
 32.0,
 60

In [23]:
import pandas

In [24]:
df1=pandas.read_csv('/root/autodl-tmp/msc_ml/llm_thought/Evaluation/test_zeroshot_base_svamp.csv')
df2=pandas.read_csv('/root/autodl-tmp/msc_ml/llm_thought/Evaluation/test_zeroshot_new_svamp.csv')


In [28]:
sum(df1['Response']==df2['Response'])

61

In [27]:
print(df1['Response'][0])
print("*********")


print(df2['Response'][0])

  Sure, I'd be happy to help! Here are my thoughts on the question:
Thoughts:
1. Let's start by identifying the given information:
* Mary has already added 5 cups of flour.
* The recipe calls for 6 cups of flour.
* The recipe also calls for 8 cups of sugar.
* The recipe calls for 7 cups of salt.
2. Now, we need to find out how many more cups of sugar Mary needs to add than cups of salt. Let's do this by subtracting the amount of flour she has already added from the total amount of flour needed:
6 cups of flour - 5 cups of flour = 1 cup of flour remaining
Now, we need to find out how many cups of sugar are left to add after subtracting the amount of flour remaining from the total amount of sugar needed:
8 cups of sugar - 1 cup of flour = 7 cups of sugar remaining
3. Finally, we can see that Mary needs to add 7 cups of sugar more than cups of salt to complete the recipe.
Answer: 7

I hope this helps! Let me know if you have any questions.
*********
  Sure, I'd be happy to help! Here are 

In [29]:
sum([len(response) for response in df1['Response']])/300

469.22

In [30]:
sum([len(response) for response in df2['Response']])/300

460.31333333333333